Originally from /home/yuchen/Documents/HCPrediction/Analysis_remap.py.

In [19]:
# This is a clean script for the remapping analysis
import numpy as np
import torch
from scipy.stats import norm

# from helper import *
# from RNN_Class import *
# from IO_plot import *
import matplotlib

matplotlib.use("Agg")
import matplotlib.pyplot as plt
import glob
import scipy.signal as signal

font = {"family": "sans-serif", "sans-serif": ["Helvetica"], "size": 14}
matplotlib.rc("font", **font)
plt.rcParams.update({"text.usetex": True})

In [21]:
# Generate input/target ranging from (0,1) for sigmoid nonlinearity (i.e. removed the softmax constrain)
print("BUILDING INPUTS...")
N, TotalSteps, T = 200, 2000, 100
Path = f"../data/Ns{N}_SeqN{T}/"
X = np.zeros((np.int64(N), np.int64(TotalSteps)))  # input N*T
tmp = np.linspace(norm.ppf(0.05), norm.ppf(0.95), np.int64(TotalSteps / 2))
BellShape = norm.pdf(tmp)  # Bellshape vector
template = np.concatenate((BellShape, np.zeros(np.int64(TotalSteps / 2))))
X = np.zeros((np.int64(N), np.int64(TotalSteps)))  # time-shifting matrix
for i in np.arange(np.int64(N)):
    X[i, :] = np.roll(template, np.int64(i * (TotalSteps / N)))

Select_T = np.arange(0, TotalSteps, np.int64(TotalSteps / T), dtype=int)
tmp = np.expand_dims((X[:, Select_T].T), axis=0)
tmp = tmp / tmp.max()
X_mini = torch.tensor(tmp.astype(np.single))
fname = f"Ns{N}_SeqN{T}_1.pth.tar"
print(f"saving X_mini and Target_mini for Env 1 to {Path}{fname}")
torch.save(
    {"X_mini": X_mini, "Target_mini": X_mini},
    Path + fname,
)

figname = f"Ns{N}_SeqN{T}_1.png"
print(f"saving Env 1 input fig to {Path}{figname}")
plt.figure()
plt.imshow(X_mini[0, :, :].T.numpy())
plt.colorbar()
plt.title("input 1")
plt.tight_layout()
plt.savefig(Path + figname)
plt.close()

BUILDING INPUTS...
saving X_mini and Target_mini for Env 1 to ../data/Ns200_SeqN100/Ns200_SeqN100_1.pth.tar
saving Env 1 input fig to ../data/Ns200_SeqN100/Ns200_SeqN100_1.png


In [16]:
# Save a noisy version of env 1
# Independent random gaussian noise N~(0,activity*noise_level)
# noise_level = 0.05
# noise_level = 0.1
# noise_level = 0.2
# noise_level = 0.3
# noise_level = 0.4
noise_level = 0.5
torch.manual_seed(0)
X_noisy = X_mini + torch.normal(mean=0, std=noise_level * X_mini)
print(
    f"saving noisy version of X_mini and Target_mini for Env 1 to {Path}Ns{N}_SeqN{T}_1_{int(noise_level * 100)}per.pth.tar"
)
torch.save(
    {"X_mini": X_noisy, "Target_mini": X_noisy},
    Path + "Ns{}_SeqN{}_1_{}per.pth.tar".format(N, T, int(noise_level * 100)),
)

print(
    f"saving Env 1 input fig to {Path}Ns{N}_SeqN{T}_1_{int(noise_level * 100)}per.png"
)
plt.figure()
plt.imshow(X_noisy[0, :, :].T.numpy())
plt.colorbar()
plt.title("input 1")
plt.tight_layout()
plt.savefig(Path + "Ns{}_SeqN{}_1_{}per.png".format(N, T, int(noise_level * 100)))
plt.close()

saving noisy version of X_mini and Target_mini for Env 1 to ../Elman_SGD/Remap_predloss/N100T100_relu_fixio/Ns100_SeqN100_1_50per.pth.tar
saving Env 1 input fig to ../Elman_SGD/Remap_predloss/N100T100_relu_fixio/Ns100_SeqN100_1_50per.png


In [17]:
# Create randomly permuted environment 2
np.random.seed(2)
idx = np.random.permutation(np.arange(N))
X_new = X_mini[:, :, idx]
print(f"saving X_mini and Target_mini for Env 2 to {Path}Ns{N}_SeqN{T}_2.pth.tar")
torch.save(
    {"X_mini": X_new, "Target_mini": X_new}, Path + "Ns{}_SeqN{}_2.pth.tar".format(N, T)
)

print(f"saving Env 2 input fig to {Path}Ns{N}_SeqN{T}_2.png")
plt.figure()
plt.imshow(X_mini[0, :, :].T.numpy())
plt.colorbar()
plt.title("input 2")
plt.tight_layout()
plt.savefig(Path + "Ns{}_SeqN{}_2.png".format(N, T))
plt.close()

saving X_mini and Target_mini for Env 2 to ../Elman_SGD/Remap_predloss/N100T100_relu_fixio/Ns100_SeqN100_2.pth.tar
saving Env 2 input fig to ../Elman_SGD/Remap_predloss/N100T100_relu_fixio/Ns100_SeqN100_2.png


In [18]:
# Combine two inputs as one env
X_mini2 = torch.stack((X_mini[0, :, :], X_new[0, :, :]), dim=0)
print(f"Combining Env 1 and Env 2. Saving to Ns{N}_SeqN{T}_2Batch.pth.tar")
torch.save(
    {"X_mini": X_mini2, "Target_mini": X_mini2},
    Path + "Ns{}_SeqN{}_2Batch.pth.tar".format(N, T),
)

Combining Env 1 and Env 2. Saving to Ns100_SeqN100_2Batch.pth.tar


Output

In [ ]:
Path = "Elman_SGD/Remap_predloss/"
N, SeqN, HN = 100, 100, 200
RecordN = 2000
data_name = "Ns{}_SeqN{}_1".format(N, SeqN)
model_name = "Ns{}_SeqN{}_predloss_full".format(N, SeqN)
model = ElmanRNN_tp1(N, HN, N)
model.act = nn.Sigmoid()
model_remap = ElmanRNN_tp1(N, HN, N)
model_remap.act = nn.Sigmoid()
net = torch.load(Path + model_name + ".pth.tar", map_location="cuda:0")
data = torch.load(Path + data_name + ".pth.tar")
model.load_state_dict(net["state_dict"])
X_mini = data["X_mini"][:, :-1, :]
Target_mini = data["Target_mini"][:, 1:, :]
output_previous, _ = model(X_mini, torch.zeros((1, 1, HN)))